In [2]:
import pandas as pd
import numpy as np
import requests
import re
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import json
import time

In [30]:
URL = 'https://novosibirsk.cian.ru/'

response = requests.get(URL)
link_list = []

# Создаем объект BeautifulSoup и указываем парсер
soup = BeautifulSoup(response.text, 'html.parser')

for i in range(5):
    # Находим все объявления о квартирах на странице
    apartments = soup.find_all('div', {'data-name': 'LinkArea'})
    button_list = soup.find_all('a', class_='_93444fe79c--button--KVooB _93444fe79c--link-button--ujZuh _93444fe79c--M--I5Xj6 _93444fe79c--button--WChcG')
    next_page_link = button_list[0].get('href')

    # Перебираем найденные объявления и вытаскиваем нужные данные
    for elem in apartments:
        link = elem.find_all('a', class_='_93444fe79c--link--eoxce')
        for i in link: 
            link_list.append(i.get('href'))
    
    response = requests.get(next_page_link)
    soup = BeautifulSoup(response.text, 'html.parser')

IndexError: list index out of range

In [3]:
cookies = {
    '_CIAN_GK': '585e86ba-621f-466e-8e82-ba8375b9c609',
    '_gcl_au': '1.1.671780311.1704126706',
    'tmr_lvid': 'c81f323f127b00fa6c688d3d57bbc73d',
    'tmr_lvidTS': '1704126706501',
    'uxfb_usertype': 'searcher',
    'uxs_uid': '42a60650-a8c3-11ee-bbfe-1b62e987354b',
    '_ym_uid': '1704126709584908912',
    '_ym_d': '1704126709',
    'afUserId': 'fd2e771f-c5c9-491e-a7a2-b28210afe858-p',
    'adrcid': 'AenYwjWuJ-O4dVzTRXJsnZg',
    'cookie_agreement_accepted': '1',
    'countCallNowPopupShowed': '0%3A1704126744099',
    'jview': '1',
    'newobject_scount': '1',
    'newobject_all': '1',
    '_gpVisits': '{"isFirstVisitDomain":true,"idContainer":"1000252B"}',
    'sopr_utm': '%7B%22utm_source%22%3A+%22direct%22%2C+%22utm_medium%22%3A+%22None%22%7D',
    'AF_SYNC': '1707236314259',
    '_gid': 'GA1.2.1219593440.1707544727',
    '_ym_isad': '2',
    'session_region_id': '4897',
    'session_main_town_region_id': '4897',
    'login_mro_popup': '1',
    '__cf_bm': 'tODHc75.9N9Z_cgsikqw4pQ4P26TVrDfCyyOC2gPwfU-1707591111-1-AQMrWMhkPqtQZJ+IpvyDlLZ7NBIL1BAafFWL9209MilhiP2G/oHXm45Yl+vKC+W125iJq5jDH9zVtXDWyeJ0g7A=',
    'anti_bot': '"2|1:0|10:1707591241|8:anti_bot|44:eyJyZW1vdGVfaXAiOiAiMzcuMTk0LjE5Ny4xMTIifQ==|54a9c9e7dd346fb0ec6a6c9003004d98a06d67c4d24b49db89229011204d30ec"',
    'sopr_session': 'eb0538b77d7d4fd3',
    '_ym_visorc': 'b',
    '_gp1000252B': '{"hits":4,"vc":1,"ac":1,"a6":1}',
    '_ga': 'GA1.2.2044989704.1704126708',
    '_ga_3369S417EL': 'GS1.1.1707591167.11.1.1707591677.60.0.0',
}

headers = {
    'authority': 'api.cian.ru',
    'accept': '*/*',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'content-type': 'application/json',
    # 'cookie': '_CIAN_GK=585e86ba-621f-466e-8e82-ba8375b9c609; _gcl_au=1.1.671780311.1704126706; tmr_lvid=c81f323f127b00fa6c688d3d57bbc73d; tmr_lvidTS=1704126706501; uxfb_usertype=searcher; uxs_uid=42a60650-a8c3-11ee-bbfe-1b62e987354b; _ym_uid=1704126709584908912; _ym_d=1704126709; afUserId=fd2e771f-c5c9-491e-a7a2-b28210afe858-p; adrcid=AenYwjWuJ-O4dVzTRXJsnZg; cookie_agreement_accepted=1; countCallNowPopupShowed=0%3A1704126744099; jview=1; newobject_scount=1; newobject_all=1; _gpVisits={"isFirstVisitDomain":true,"idContainer":"1000252B"}; sopr_utm=%7B%22utm_source%22%3A+%22direct%22%2C+%22utm_medium%22%3A+%22None%22%7D; AF_SYNC=1707236314259; _gid=GA1.2.1219593440.1707544727; _ym_isad=2; session_region_id=4897; session_main_town_region_id=4897; login_mro_popup=1; __cf_bm=tODHc75.9N9Z_cgsikqw4pQ4P26TVrDfCyyOC2gPwfU-1707591111-1-AQMrWMhkPqtQZJ+IpvyDlLZ7NBIL1BAafFWL9209MilhiP2G/oHXm45Yl+vKC+W125iJq5jDH9zVtXDWyeJ0g7A=; anti_bot="2|1:0|10:1707591241|8:anti_bot|44:eyJyZW1vdGVfaXAiOiAiMzcuMTk0LjE5Ny4xMTIifQ==|54a9c9e7dd346fb0ec6a6c9003004d98a06d67c4d24b49db89229011204d30ec"; sopr_session=eb0538b77d7d4fd3; _ym_visorc=b; _gp1000252B={"hits":4,"vc":1,"ac":1,"a6":1}; _ga=GA1.2.2044989704.1704126708; _ga_3369S417EL=GS1.1.1707591167.11.1.1707591677.60.0.0',
    'origin': 'https://novosibirsk.cian.ru',
    'referer': 'https://novosibirsk.cian.ru/',
    'sec-ch-ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
}

# Note: json_data will not be serialized by requests
# exactly as it was in the original request.
data = '{"jsonQuery":{"_type":"flatsale","engine_version":{"type":"term","value":2},"region":{"type":"terms","value":[4897]},"page":{"type":"term","value":7},"room":{"type":"terms","value":[1,2,3,4,5,6,7,9]}},"offerIds":[290330777,297269829,298317658,296728943,290158748,286641705,295755124,297729572,290330746,287410209,287548436,297773735,287548362,297376576,294434785,290935385,279447250,294784309,297138027,297326705,291188048,296405777,286239641,297472014,294607709,285391472,297161985,287217073],"pageNumber":1,"queryString":"deal_type=sale&engine_version=2&offer_type=flat&p=7&region=4897&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1"}'
response = requests.post(
    'https://api.cian.ru/search-offers/v1/get-infinite-search-result-desktop/',
    cookies=cookies,
    headers=headers,
    data=data,
)

result =response.json()

In [4]:
link_list = []

for i in range(len(result['infiniteSearchResult'])):
    link = 'https://novosibirsk.cian.ru/sale/flat/'
    id = str(result['infiniteSearchResult'][i]['itemId']) + '/'
    link += id
    link_list.append(link)

In [30]:
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36')
options.add_argument('--disable-blink-features=AutomationControlled')

# Создание драйвера
driver = webdriver.Chrome(options=options)
flat_list = []

for i in link_list[link_list.index('https://novosibirsk.cian.ru/sale/flat/296531341/'):]:
    flat = {}
    price = ''
    square = ''
    living_square = ''
    kitchen = ''
    year = ''
    decor = ''
    address = ''
    floor = ''
    building_type = ''
    ceil_height = ''
    elevators = ''
    metro = ''

    driver.get(i)
    
    address = driver.find_element(By.XPATH, '//*[@id="frontend-offer-card"]/div/div[2]/div[2]/section/div/div/div[2]/address/div/div')
    metro = driver.find_elements(By.CLASS_NAME, 'a10a3f92e9--underground--pjGNr')

    html = driver.find_element(By.TAG_NAME, "html")
    time.sleep(0.05)

    for k in range(50):
        html.send_keys(Keys.DOWN)

    try:
        price = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div[3]/div/div[1]/div[1]/div[3]/div/div[1]/span')
    except:
        price = driver.find_element(By.XPATH, '//*[@id="frontend-offer-card"]/div/div[2]/div[3]/div/div[1]/div[1]/div[4]/div/div[1]/span')
    square = driver.find_elements(By.XPATH, '//*[@id="frontend-offer-card"]/div/div[2]/div[2]/div[5]/div/div/div/div[1]/div[1]/div[2]/div[2]/span[2]')
    living_square = driver.find_element(By.CLASS_NAME, 'a10a3f92e9--color_black_100--Ephi7 a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_normal--JEG_c a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10a3f92e9--text--e4SBY a10a3f92e9--text_letterSpacing__0--cQxU5')

    try:
        kitchen = driver.find_element(By.CLASS_NAME, 'a10a3f92e9--color_black_100--Ephi7 a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_normal--JEG_c a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10a3f92e9--text--e4SBY a10a3f92e9--text_letterSpacing__0--cQxU5')
    except:
        kitchen = None

    try:
        year = driver.find_element(By.CLASS_NAME, 'a10a3f92e9--color_black_100--Ephi7 a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_bold--BbhnX a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10a3f92e9--text--e4SBY')
    except:
        year = driver.find_element(By.XPATH, '//*[@id="frontend-offer-card"]/div/div[2]/div[2]/div[3]/div[3]/div[2]/span[2]')

    decor = driver.find_element(By.CLASS_NAME, 'a10a3f92e9--color_black_100--Ephi7 a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_normal--JEG_c a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10a3f92e9--text--e4SBY a10a3f92e9--text_letterSpacing__0--cQxU5')
    
    floor = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div[2]/div[3]/div[4]/div[2]/span[2]')
    
    try:
        ceil_height = driver.find_element(By.CLASS_NAME, 'a10a3f92e9--color_black_100--Ephi7 a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_normal--JEG_c a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10a3f92e9--text--e4SBY a10a3f92e9--text_letterSpacing__0--cQxU5')
    except:
        ceil_height = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div[2]/div[6]/div/div/div/div[1]/div[1]/div[2]/div[3]/span[2]')

    try:
        elevators = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div[2]/div[5]/div/div/div/div[1]/div[2]/div[2]/div[1]/span[2]')
    except:
        elevators = None
    

    flat['price'] = price.text
    flat['square'] = square.text
    flat['living_square'] = living_square.text

    try:
        flat['kitchen']= kitchen.text
    except:
        flat['kitchen']= None

    flat['year'] = year.text
    #flat['decor'] = decor.text
    flat['address'] = address.text
    flat['floor'] = floor.text
    #flat['building_type'] = building_type.text
    flat['ceil_height'] = ceil_height.text

    try:
        flat['elevators'] = elevators.text
    except AttributeError:
        flat['elevators'] = None
    #flat['metro'] = metro.text

    flat_list.append(flat)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".a10a3f92e9--color_black_100--Ephi7 a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_normal--JEG_c a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10a3f92e9--text--e4SBY a10a3f92e9--text_letterSpacing__0--cQxU5"}
  (Session info: chrome=121.0.6167.161); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF79C595E42+3538674]
	(No symbol) [0x00007FF79C1B4C02]
	(No symbol) [0x00007FF79C065AEB]
	(No symbol) [0x00007FF79C0ABF4E]
	(No symbol) [0x00007FF79C0AC0CC]
	(No symbol) [0x00007FF79C0EE477]
	(No symbol) [0x00007FF79C0CF0EF]
	(No symbol) [0x00007FF79C0EBDE3]
	(No symbol) [0x00007FF79C0CEE53]
	(No symbol) [0x00007FF79C09F514]
	(No symbol) [0x00007FF79C0A0631]
	GetHandleVerifier [0x00007FF79C5C6CAD+3738973]
	GetHandleVeri

In [26]:
for i in flat_list:
    print(i)

{'price': '9 900 000 ₽', 'square': '86,04 м²', 'living_square': '86,04 м²', 'kitchen': '2023', 'year': 'Предчистовая', 'address': 'Новосибирская область, Новосибирск, р-н Заельцовский, мкр. Аэропорт, ул. Аэропорт, 88На карте', 'floor': 'Сдан', 'ceil_height': '2,72 м', 'elevators': '2 пассажирских, 1 грузовой'}
{'price': '8 900 000 ₽', 'square': '98 м²', 'living_square': '98 м²', 'kitchen': '10 м²', 'year': '2017', 'address': 'Новосибирская область, Новосибирск, р-н Октябрьский, ул. Декабристов, 117На карте', 'floor': '17 из 17', 'ceil_height': '83', 'elevators': None}
{'price': '9 250 000 ₽', 'square': '63,81 м²', 'living_square': '63,81 м²', 'kitchen': '24 из 25', 'year': 'Не сдан', 'address': 'Новосибирская область, Новосибирск, р-н Дзержинский, ул. Гоголя, 222На карте', 'floor': '2025', 'ceil_height': '42,1', 'elevators': '2 грузовых'}
{'price': '9 300 000 ₽', 'square': '63,44 м²', 'living_square': '63,44 м²', 'kitchen': '5,1 м²', 'year': '2026', 'address': 'Новосибирская область, Н

In [ ]:
<span class="a10a3f92e9--color_black_100--Ephi7 a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_normal--JEG_c a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10a3f92e9--text--e4SBY a10a3f92e9--text_letterSpacing__0--cQxU5">83</span>
building_type = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div[2]/div[5]/div/div/div/div[1]/div[2]/div[2]/div[2]/span[2]')
except NoSuchElementException:
    building_type = driver.find_element(By.XPATH, '//*[@id="frontend-offer-card"]/div/div[2]/div[2]/div[6]/div/div/div/div[1]/div[2]/div[2]/div/span[2]')